In [18]:
!pip install transformers
!pip install accelerate -U


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [19]:
!pip install --upgrade jupyterlab ipywidgets


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [20]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [21]:
# Example input text (natural language instruction)
input_text = "Create a Python function that prints Hello World!"

# Tokenize the input
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate the output (command)
outputs = model.generate(input_ids,max_new_tokens=50)

# Decode and print the generated command
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Create a Python function that prints Hello World!


In [22]:
import pandas as pd

# Load the dataset
dataset = pd.read_csv('text_to_command_dataset.csv')

# Preview the dataset
print(dataset.head())

                           input_text                            output_text
0        create a cube in front of me   cube-create-relative 100 0 0 10 gray
1  create a black cube in front of me  cube-create-relative 100 0 0 10 black
2   create a blue cube in front of me   cube-create-relative 100 0 0 10 blue
3  create a green cube in front of me  cube-create-relative 100 0 0 10 green
4    create a red cube in front of me    cube-create-relative 100 0 0 10 red


In [23]:
from datasets import Dataset
from transformers import T5Tokenizer

# Convert pandas dataframe to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(dataset)

# Tokenize input and output
def preprocess_function(examples):
    inputs = [ex for ex in examples['input_text']]
    targets = [ex for ex in examples['output_text']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the tokenizer
tokenized_dataset = hf_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [24]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Start fine-tuning
trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`